#### Random Graph Generation

In [1]:
import numpy as np
import random
import pickle
import os

K=[10]
nV=[150]                      #nv is the number of vertices in the graph.  
nC=[15]                      #cv is the number of clients for each facility
num_graphs=5

penalty=10000
Fac_Open_Cost = 1000

modes=['skewed[1-10]']
da=['training','testing']
# mode='/skewed[1-6]'
# mode='/skewed[1,10]'
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+mode)
            os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc))
            for ki,k in enumerate(K):
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k))
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing')
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training')
                print(nv,nc,mode,k)
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for graph in range(num_graphs):
                        adj=np.zeros([nv*nc,3])
                        new_pairs=[]
                        for i in range(1,nv+1):
                            y=np.sort(random.sample(range(1,nv+1), nc))
                            for j in (y):
                                new_pairs.append((i,j))
                        new_pairs=np.array(new_pairs)


                        adj[:,0]=new_pairs[:,0]
                        adj[:,1]=new_pairs[:,1]
                        #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                        #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                        adj[:,2]= random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                        #adj[:,2]= random.choices([1,100], k = nc*nv)

                        os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv,k],open(path+str(graph)+'/data','wb'))
                        np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc)+'   '+str(k))

150 15 skewed[1-10] 10


#### LP for feature computation

In [2]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,data,c,n,k):
    print("Inside kmedian",n,k)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(c*x)+xp.Sum(y)*Fac_Open_Cost)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

#### Computing LP based features

In [3]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

#### Computing features not based on LP

In [4]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=penalty)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

#### Calling feature computation (LP-based and non LP-based) after computing LP

In [5]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for k in K:
                print('------------------------------------------')
                for d in da:
                    print('--- nV=',nv,' nC=',nc,' k=',k, ' mode=',d)
                    for graph in range(num_graphs):
                        print('       data=',graph)
                        Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'+str(graph)
                        data = pickle.load(open(Mypath+'/data','rb'))
                        nv= data[1]; k=data[2]; data=data[0]
                        data=data.astype(int)
                        data[:,0]=data[:,0]-1
                        data[:,1]=data[:,1]-1
                        cost_mtr=np.zeros([nv,nv])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]
                        #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                        median_indices,x,y=kmedian_xpress_solver_LP(Mypath,data,cost_mtr,nv,k)
                        features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                        features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
------------------------------------------
--- nV= 150  nC= 15  k= 10  mode= training
       data= 0
Inside kmedian 150 10
(150, 150) (150,)
FICO Xpress v8.10.0, Hyper, solve started 0:08:33, Aug 16, 2021
Heap usage: 11MB (peak 11MB, 1073KB system)
Minimizing LP noname
Original problem has:
     22650 rows        22650 cols        67500 elements
Presolved problem has:
     22650 rows        22650 cols        67500 elements
Presolve finished in 0 seconds
Heap usage: 14MB (peak 22MB, 1074KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Starting parallel dual simplex, using up to 8 threads
 
   Its      

  2654       12731.28917      P      0     0        .000000     1
Uncrunching matrix
Optimal solution found
Dual solved problem
  2654 simplex iterations in 1s

Final objective                       : 1.273128916794253e+04
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
            Time:  1.5561419711448252
            objective value:  12731.289167942576

       data= 1
Inside kmedian 150 10
(150, 150) (150,)
FICO Xpress v8.10.0, Hyper, solve started 0:08:45, Aug 16, 2021
Heap usage: 11MB (peak 11MB, 1113KB system)
Minimizing LP noname
Original problem has:
     22650 rows        22650 cols        67500 elements
Presolved problem has:
     22650 rows        22650 cols        67500 elements
Presolve finished in 0 seconds
Heap usage: 14MB (peak 22MB, 1114KB system)

Coefficient range                    original                 solved        
  Coeffic

#### Run ILP to get labelled training data and evaluation on test data

In [ ]:
import xpress as xp
from termcolor import colored
import timeit
import math
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import networkx as nx

def floyd_warshall(G,nV):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    # Adding vertices individually
    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    return (distance)

def kmedian_xpress_solver_ILP(data,n,k):

    c=np.zeros([n,n])+penalty
    data=data.astype(int)
    data[:,0]=data[:,0]-1
    data[:,1]=data[:,1]-1
    for i in range (len(data)):
        c[data[i,1],data[i,0]]=data[i,2]
    #c=floyd_warshall(c,n)

    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in S])

    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    profit=xp.Sum(xp.Sum(c*x)+xp.Sum(y)*Fac_Open_Cost)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c2)
    p.addConstraint(c3)
    p.solve()

    median_indices=np.where(p.getSolution(y)>=.5)[0]
    return median_indices,p.getSolution(profit)


for nv in nV:
    for nc in nC:
        for k in K:
            for mode in modes:
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for m in range(num_graphs):
                        print(colored('....................................................','blue'),m)
                        S = timeit.default_timer()
                        set_path=path+str(m)
                        data = pickle.load(open(set_path+'/data','rb'))
                        #print(data[0],data[1],data[2])

                        labels=kmedian_xpress_solver_ILP(data[0],data[1],data[2])

                        E = timeit.default_timer() 
                        pickle.dump([labels,E-S],open(set_path+'/points_inf_xp','wb'))

                        print(data[1],data[2],data[0].shape)
                        print(colored('objective value: ','red'),labels[1])
                        print(E - S) 
                        print(colored('median indices: ','red'),labels[0]+1,len(labels[0]))

.................................................... 0
FICO Xpress v8.10.0, Hyper, solve started 0:08:53, Aug 16, 2021
Heap usage: 11MB (peak 11MB, 5474KB system)
Minimizing MILP noname
Original problem has:
     22650 rows        22650 cols        67500 elements     22650 globals
Presolved problem has:
     22650 rows        22650 cols        67500 elements     22650 globals
Presolve finished in 0 seconds
Heap usage: 18MB (peak 27MB, 5476KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 88.3GB
 *** Heuristic solution found:  1351094.000      Time: 0 ***
 *** Heuristic solution found:  24823.00000      Time: 0 ***
Starting con

    2310  18863.00000  12945.22993     11   1739     50   31.37%     817     34
    2410  18863.00000  12945.22993     11   1896     37   31.37%     628     34
    2513  18863.00000  12945.22993     11   1948     41   31.37%     639     34
    2616  18863.00000  12945.22993     11   2079     64   31.37%     406     34
    2716  18863.00000  12945.22993     11   2105     33   31.37%     616     34
    2816  18863.00000  12945.22993     11   2210     50   31.37%     712     35
    2916  18863.00000  12945.22993     11   2332     70   31.37%     619     35
    3018  18863.00000  12945.22993     11   2380     63   31.37%     613     35
Resetting tree to root.

Performing root presolve...

Reduced problem has:    2415 rows    2400 columns     10424 elements
Presolve dropped   :       0 rows       0 columns     11697 elements
Will try to keep branch and bound tree memory usage below 65.4GB
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
1717667       11950.57337      P   

    8328  18863.00000  12945.22993     11   4055     49   31.37%     357     66
    8433  18863.00000  12945.22993     11   4131     66   31.37%     401     66
    8534  18863.00000  12945.22993     11   4274     60   31.37%     371     67
    8636  18863.00000  12945.22993     11   4386     63   31.37%     503     67
    8736  18863.00000  12945.22993     11   4464     19   31.37%     818     67
    8839  18863.00000  12945.22993     11   4575     18   31.37%     836     67
    8942  18863.00000  12945.22993     11   4629     28   31.37%     907     67
    9042  18863.00000  12945.22993     11   4681     17   31.37%     805     67
    9142  18863.00000  12945.22993     11   4686     39   31.37%     882     68
    9243  18863.00000  12945.22993     11   4703     18   31.37%     664     68
Heap usage: 1937MB (peak 1937MB, 22MB system)
B&B tree size: 13MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9344  18863.00000  12945.22993     11   47

   55692  16901.00000  13133.33491     20  24180     12   22.29%     821    120
   56692  16901.00000  13140.81763     20  24907     19   22.25%     911    121
   57692  16901.00000  13140.81763     20  25449     18   22.25%     846    123
   58694  16901.00000  13140.81763     20  25996     31   22.25%     744    124
   59698  16901.00000  13160.46275     20  26658     49   22.13%     710    126
   60701  16901.00000  13160.46275     20  27080     32   22.13%     452    127
   61701  16901.00000  13160.46275     20  27801     48   22.13%     577    128
   62703  16901.00000  13183.28820     20  28330     19   22.00%     735    130
   63707  16901.00000  13183.28820     20  28688     39   22.00%     678    131
   64707  16901.00000  13183.28820     20  29515     28   22.00%     745    133
   65709  16901.00000  13186.07875     20  29905     42   21.98%     703    134
   66709  16901.00000  13186.07875     20  30596     40   21.98%     800    136
   67709  16901.00000  13207.23381     2

  553839  16863.00000  15033.81991     22 251900     33   10.85%     530    654
  563839  16863.00000  15040.03069     22 255863     46   10.81%     692    663
Heap usage: 2888MB (peak 2888MB, 22MB system)
B&B tree size: 0.8GB total
               8.3MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
  573840  16863.00000  15045.15193     22 259890     31   10.78%     799    674
  583841  16863.00000  15046.46053     22 263804     19   10.77%     622    683
  593843  16863.00000  15059.32415     22 267740     29   10.70%     676    693
  603846  16863.00000  15065.35364     22 271692     31   10.66%     576    703
  613846  16863.00000  15071.88427     22 275627     29   10.62%     658    713
  623847  16863.00000  15079.45353     22 279517     28   10.58%     770    722
  633847  16863.00000  15084.68342     22 283346     40   10.55%     560    732
  643849  16863.00000  15092.18415     22 287080     31   10.50%     696    742


...

 14440034  16827.00000  16412.14802     24 408042     34    2.47%     639  13653
 14450036  16827.00000  16413.70419     24 404744     23    2.46%     537  13663
 14460036  16827.00000  16415.47682     24 401308     33    2.45%     654  13673
 14470039  16827.00000  16417.09863     24 397930     37    2.44%     616  13683
 14480039  16827.00000  16418.79581     24 394572     52    2.43%     551  13693
Heap usage: 8720MB (peak 10GB, 22MB system)
B&B tree size: 6.4GB total
               38MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 14490041  16827.00000  16420.54608     24 391162     33    2.42%     559  13703
 14500044  16827.00000  16421.99296     24 387749     27    2.41%     683  13713
 14510044  16827.00000  16423.78919     24 384517     33    2.40%     611  13723
 14520044  16827.00000  16425.35037     24 381171     38    2.39%     660  13733
 14530044  16827.00000  16427.30604     24 377781     30    2.38%     656 

 15330157  16827.00000  16710.07296     24  63748     31    0.69%     435  14399
 15340157  16827.00000  16717.64914     24  57940     42    0.65%     541  14404
 15350159  16827.00000  16725.66139     24  52072     32    0.60%     638  14409
 15360161  16827.00000  16733.89174     24  45954     34    0.55%     720  14415
 15370161  16827.00000  16742.20154     24  39562     65    0.50%     640  14420
 15380161  16827.00000  16753.01312     24  32930     37    0.44%     585  14425
 15390161  16827.00000  16763.57428     24  26106     37    0.38%     527  14430
 15400163  16827.00000  16775.92244     24  18906     29    0.30%     520  14435
 15410164  16827.00000  16792.07416     24  11049     31    0.21%     669  14440
 15420164  16827.00000  16815.80769     24   2271     28    0.07%     543  14446
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.682700000000000e+04
Final MIP bound                       : 1.682700000000000e+04
  Solution time / pri

    1301  17887.00000  13024.69387      6   1156     76   27.18%     544     24
    1401  17887.00000  13073.39034      6   1205     20   26.91%     872     25
    1501  17887.00000  13073.39034      6   1205     26   26.91%     740     25
    1602  17887.00000  13073.39034      6   1205     26   26.91%     696     25
    1705  17887.00000  13073.39034      6   1216     28   26.91%     685     25
    1807  17887.00000  13073.39034      6   1252     26   26.91%     725     26
    1907  17887.00000  13073.39034      6   1310     48   26.91%     615     26
    2009  17887.00000  13073.39034      6   1445     51   26.91%     644     26
    2111  17887.00000  13073.39034      6   1660     55   26.91%     539     26
Heap usage: 1059MB (peak 1059MB, 15MB system)
B&B tree size: 4.5MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    2212  17887.00000  13073.39034      6   1803     44   26.91%     704     26
    2315  17887.00000  13073.39034      6   1

    7072  17887.00000  13103.70359      6   3226     27   26.74%     793     71
    7172  17887.00000  13103.70359      6   3245     26   26.74%     717     72
    7272  17887.00000  13103.70359      6   3255     24   26.74%     729     72
    7372  17887.00000  13103.70359      6   3263     26   26.74%     799     72
Heap usage: 1866MB (peak 1866MB, 22MB system)
B&B tree size: 9.5MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    7474  17887.00000  13186.59138      6   3371     27   26.28%     799     73
    7575  17887.00000  13186.59138      6   3450     31   26.28%     799     73
    7675  17887.00000  13186.59138      6   3475     29   26.28%     836     73
    7776  17887.00000  13186.59138      6   3524     16   26.28%     581     73
    7879  17887.00000  13186.59138      6   3533     40   26.28%     715     74
    7981  17887.00000  13186.59138      6   3611     28   26.28%     665     74
    8082  17887.00000  13186.59138      6   3

   41413  17841.00000  13419.78014      9  25802     22   24.78%     611    127
   42413  17841.00000  13490.78498      9  26239     29   24.38%     800    128
   43416  17841.00000  13490.78498      9  26970     53   24.38%     519    130
   44416  17841.00000  13490.78498      9  27753     29   24.38%     776    131
   45417  17841.00000  13490.78498      9  28131     50   24.38%     481    133
   46418  17841.00000  13532.25089      9  28868     19   24.15%     783    134
   47420  17841.00000  13532.25089      9  29651     38   24.15%     583    135
   48423  17841.00000  13532.25089      9  30170     34   24.15%     708    137
   49423  17841.00000  13549.52465      9  30882     40   24.05%     646    138
   50423  17841.00000  13549.52465      9  31627     28   24.05%     661    140
   51423  17841.00000  13549.52465      9  32064     41   24.05%     530    142
Heap usage: 2002MB (peak 2002MB, 22MB system)
B&B tree size: 103MB total
 
    Node     BestSoln    BestBound   Sols Act

#### Load libraries to run Classification model

In [1]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_curve, confusion_matrix, accuracy_score
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve
from sklearn import svm

#### Load features for classification

In [2]:
def feature_loader(set_path): 
    f0=pickle.load(open(set_path+'/alfa_f','rb'))
    f1=pickle.load(open(set_path+'/B_f_2','rb'))
    f2=pickle.load(open(set_path+'/B_f_4','rb'))
    f3=pickle.load(open(set_path+'/B_f_8','rb'))
    f4=pickle.load(open(set_path+'/lp_kmedian','rb'))
    f5=pickle.load(open(set_path+'/min_f','rb'))
    f6=pickle.load(open(set_path+'/V_f_2','rb'))
    f7=pickle.load(open(set_path+'/V_f_4','rb'))
    f8=pickle.load(open(set_path+'/V_f_8','rb'))
    f9=pickle.load(open(set_path+'/yf','rb'))
    f10=pickle.load(open(set_path+'/Yf','rb'))
    f11=pickle.load(open(set_path+'/µf','rb'))
    f12=pickle.load(open(set_path+'/Yfp','rb'))
#     f13=pickle.load(open(set_path+'/degree','rb'))
    f14=pickle.load(open(set_path+'/T_f_2','rb'))
    f15=pickle.load(open(set_path+'/T_f_4','rb'))
    f16=pickle.load(open(set_path+'/T_f_8','rb'))
#     f17=pickle.load(open(set_path+'/min_c','rb'))
#     f18=pickle.load(open(set_path+'/f1','rb'))
#     f19=pickle.load(open(set_path+'/f2','rb'))
#     f20=pickle.load(open(set_path+'/lp_prime','rb'))
#     f21=pickle.load(open(set_path+'/NLP1','rb'))
    #f22=pickle.load(open(set_path+'/NILP','rb'))
    #f23= pickle.load(open(set_path+'/ga_ind','rb'))
    
    features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:],f12[:],f14[:],f15[:],f16[:]]);features=features.transpose()
    #features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:]]);features=features.transpose()
    #features=np.vstack([f4[:],f10[:],f7[:],f2[:]]);features=features.transpose()
    
    
    return(features)

#### Load ground truth for classification

In [6]:
path='data/'+modes[0]+'/V='+str(nV[0])+'-C='+str(nC[0])+'/k='+str(K[0])+'/training/0'
y=pickle.load(open(path+'/points_inf_xp','rb'))
y_true=np.zeros(nV[0]);y_true[y[0][0]]=1
features=feature_loader(path)


for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(1,num_graphs):
                    Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training/'+str(i)
                    y=pickle.load(open(Mypath+'/points_inf_xp','rb'))
                    y_temp=np.zeros(nV[0]);y_temp[y[0][0]]=1
                    y_true=np.hstack([y_true,y_temp])
                    f_temp=feature_loader(Mypath)
                    features=np.vstack([features,f_temp])
print(len(y_true),len(features))


750 750


#### Normalisation and Learning Classification Model

In [7]:
sc = StandardScaler()
features = sc.fit_transform(features)


clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=10, random_state=0,class_weight="balanced")
# clf = xgboost.XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0, class_weight='balanced')
# clf=AdaBoostClassifier(n_estimators=100, random_state=0)
# clf=svm.SVC(cache_size=200, class_weight='balanced',random_state=None)

clf.fit(features,y_true)

print(colored('Feature importance:','blue'))
print(clf.feature_importances_)

Feature importance:
[0.17965645 0.07601116 0.01936647 0.02038481 0.01323392 0.05343027
 0.02766364 0.0102238  0.01755705 0.16913244 0.04680926 0.10667884
 0.04717536 0.0523992  0.06820018 0.09207715]


#### Output the learnt model to a file

In [8]:
for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(num_graphs):
                    test_path = 'data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'+str(i)

                    test_features=feature_loader(test_path)
                    test_features = sc.transform(test_features)

                    y_pred = clf.predict(test_features)

                    probs= clf.predict_proba(test_features)

                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/pruning_points_'+str(i),'wb'))
                    pickle.dump(probs[:,1],open(test_path+'/probs','wb'))
                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/probs_xgb','wb'))


                    print(len(np.where(y_pred==1)[0]))

31
28
33
30
31


#### Compute the ILP on pruned data set (pruning based on different thresholds)

In [15]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_pruned(points,n,c,num_fac,k):
    S=range(n);SS=range(n*num_fac)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,num_fac)
    y = np.array([xp.var (vartype = xp.binary) for i in range(num_fac)])

    c2=xp.Dot(x,np.ones([num_fac,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,num_fac),x],axis=0))<=np.zeros([n,num_fac])

    profit=xp.Sum(xp.Sum((c*x))+xp.Sum(y)*Fac_Open_Cost)
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c2,c3)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv),min(int(6*k),nv),int(nv/2),min(int(10*k),nv),int(nv)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]
                        num_fac = len(ind_last_thre)

                        c=np.zeros([n,len(ind_last_thre)])
                        for i in range(n):
                            for j in range(len(ind_last_thre)):
                                    c[i,j]=penalty

                        for i in range (len(ind_last_thre)):
                            mp=(np.where(data[:,0]==ind_last_thre[i])[0])
                            c[data[mp,1],i]=data[mp,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_pruned(data,n,c,num_fac,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(ind_last_thre[(median_indices)])+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                        s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                        s=s[0][1]
                        value_n[th_index]=value/s
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))

.................................................... 1
FICO Xpress v8.10.0, Hyper, solve started 18:02:07, Aug 17, 2021
Heap usage: 11MB (peak 11MB, 1444KB system)
Minimizing MILP noname
Original problem has:
     22650 rows        22650 cols        67500 elements     22650 globals
Presolved problem has:
     22650 rows        22650 cols        67500 elements     22650 globals
Presolve finished in 0 seconds
Heap usage: 18MB (peak 27MB, 1445KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 61.6GB
 *** Heuristic solution found:  1351113.000      Time: 0 ***
 *** Heuristic solution found:  25818.00000      Time: 0 ***
Starting co

    2808  19868.00000  13239.85329      9   2344     62   33.36%     479     14
    2910  19868.00000  13239.85329      9   2344     58   33.36%     560     14
    3012  19868.00000  13239.85329      9   2396     62   33.36%     501     14
q   3112  19867.00000  13239.85329     10   2396     73   33.36%       0     14
    3115  19867.00000  13239.85329     10   2396     75   33.36%     219     14
    3217  19867.00000  13239.85329     10   2516     68   33.36%     411     15
    3320  19867.00000  13239.85329     10   2589     61   33.36%     599     15
Resetting tree to root.

Performing root presolve...

Reduced problem has:    2413 rows    2400 columns     10006 elements
Presolve dropped   :      92 rows      92 columns     10462 elements
Will try to keep branch and bound tree memory usage below 61.5GB
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
1866413       12616.48132      P      0     0        .000000    15
Optimal solution found
Dual solved problem
  186

    8640  17892.00000  13239.85329     11   4172     39   26.00%     551     47
    8740  17892.00000  13239.85329     11   4256     39   26.00%     491     47
    8840  17892.00000  13239.85329     11   4364     26   26.00%     807     48
    8940  17892.00000  13239.85329     11   4442     12   26.00%     858     48
    9041  17892.00000  13239.85329     11   4575     25   26.00%     980     48
    9143  17892.00000  13239.85329     11   4703     54   26.00%     540     48
    9243  17892.00000  13239.85329     11   4776     41   26.00%     709     48
    9343  17892.00000  13239.85329     11   4830     23   26.00%     978     48
Heap usage: 1650MB (peak 1650MB, 17MB system)
B&B tree size: 14MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9446  17892.00000  13239.85329     11   4863     74   26.00%     642     48
    9546  17892.00000  13239.85329     11   4950     69   26.00%     663     49
    9647  17892.00000  13239.85329     11   50

   57450  17845.00000  13510.44502     13  40458     36   24.29%     589    115
   58453  17845.00000  13564.56041     13  41126     64   23.99%     602    116
   59455  17845.00000  13564.56041     13  41878     56   23.99%     602    117
a  60392  16952.00000  13564.56041     14  42542     45   19.98%       0    119
   70020  16952.00000  13564.56041     14  32937      1   19.98%       1    119
   71021  16952.00000  13564.56041     14  33475     54   19.98%     729    120
   72023  16952.00000  13602.16772     14  34092     53   19.76%     565    121
   73023  16952.00000  13602.16772     14  34835     22   19.76%     711    123
   74023  16952.00000  13668.80808     14  35365     44   19.37%     734    124
   75023  16952.00000  13668.80808     14  36017     28   19.37%     850    126
   76023  16952.00000  13668.80808     14  36733     42   19.37%     632    127
   77024  16952.00000  13711.26799     14  37248     36   19.12%     820    128
   78028  16952.00000  13711.26799     1

Heap usage: 3202MB (peak 3202MB, 17MB system)
B&B tree size: 1.4GB total
               10MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
  653813  16898.00000  15290.60274     17 324373     37    9.51%     718    753
  663814  16898.00000  15293.48611     17 328746     27    9.50%     596    763
  673815  16898.00000  15303.53364     17 332990     26    9.44%     784    773
  683815  16898.00000  15308.38189     17 337200     33    9.41%     876    783
  693818  16898.00000  15311.49097     17 341376     55    9.39%     543    792
  703818  16898.00000  15317.06027     17 345489     24    9.36%     579    802
  713820  16898.00000  15323.44566     17 349664     51    9.32%     551    812
  723825  16898.00000  15329.05909     17 353839     66    9.28%     685    822
  733827  16898.00000  15334.44069     17 357841     40    9.25%     589    831
  743827  16898.00000  15341.31823     17 361969     35    9.21%     575    841
 

...

 6966516  16823.00000  16253.41674     26 928406     38    3.39%     694   5560
Heap usage: 10GB (peak 10GB, 17MB system)
B&B tree size: 8.2GB total
               31MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 6976516  16823.00000  16254.38402     26 927385     48    3.38%     611   5566
 6986518  16823.00000  16255.22934     26 926287     24    3.37%     434   5573
 6996519  16823.00000  16256.15339     26 925189     46    3.37%     646   5580
 7006519  16823.00000  16257.07313     26 924108     44    3.36%     616   5587
 7016520  16823.00000  16258.25265     26 923006     39    3.36%     633   5594
 7026522  16823.00000  16259.10820     26 921936     25    3.35%     598   5600
 7036523  16823.00000  16260.03803     26 920829     42    3.35%     603   5607
 7046528  16823.00000  16260.69806     26 919778     28    3.34%     524   5614
 7056531  16823.00000  16262.03928     26 918666     29    3.33%     678   5620
 7066

 7876679  16823.00000  16343.08494     26 805818     36    2.85%     632   6169
 7886679  16823.00000  16344.05166     26 804135     36    2.85%     546   6175
 7896679  16823.00000  16345.24103     26 802467     41    2.84%     439   6182
 7906680  16823.00000  16346.23804     26 800843     31    2.83%     571   6189
 7916680  16823.00000  16347.41899     26 799005     32    2.83%     467   6195
 7926685  16823.00000  16348.51884     26 797324     39    2.82%     525   6202
 7936685  16823.00000  16349.64203     26 795642     28    2.81%     498   6208
 7946691  16823.00000  16350.68128     26 793938     37    2.81%     492   6215
 7956692  16823.00000  16351.77992     26 792189     37    2.80%     534   6222
 7966693  16823.00000  16352.68787     26 790420     39    2.80%     606   6228
Heap usage: 10GB (peak 10GB, 17MB system)
B&B tree size: 8.2GB total
               31MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 7976

 8786847  16823.00000  16448.44934     26 621432     51    2.23%     477   6752
 8796848  16823.00000  16449.68347     26 619060     26    2.22%     547   6759
 8806849  16823.00000  16450.96327     26 616618     18    2.21%     577   6765
 8816850  16823.00000  16452.30556     26 614237     42    2.20%     603   6771
 8826853  16823.00000  16453.58639     26 611782     33    2.20%     664   6777
 8836855  16823.00000  16454.95722     26 609360     37    2.19%     502   6783
 8846856  16823.00000  16456.18605     26 606972     30    2.18%     648   6789
 8856856  16823.00000  16457.51605     26 604476     29    2.17%     478   6795
 8866856  16823.00000  16458.70515     26 602080     29    2.17%     569   6801
 8876858  16823.00000  16460.07913     26 599648     28    2.16%     441   6807
 8886864  16823.00000  16461.30699     26 597190     36    2.15%     571   6813
 8896864  16823.00000  16462.61769     26 594771     29    2.14%     596   6819
 8906864  16823.00000  16464.10771     2

 9717029  16823.00000  16584.17840     26 368255     35    1.42%     525   7259
 9727032  16823.00000  16585.79714     26 365118     34    1.41%     624   7264
 9737035  16823.00000  16587.68817     26 362086     26    1.40%     479   7269
 9747035  16823.00000  16589.33801     26 358956     36    1.39%     523   7273
 9757038  16823.00000  16591.06783     26 355837     39    1.38%     519   7278
 9767041  16823.00000  16592.63573     26 352762     36    1.37%     644   7282
Heap usage: 8236MB (peak 10GB, 17MB system)
B&B tree size: 6.2GB total
               31MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 9777046  16823.00000  16594.50114     26 349619     49    1.36%     647   7287
 9787046  16823.00000  16596.12628     26 346544     67    1.35%     638   7291
 9797046  16823.00000  16597.79887     26 343387     25    1.34%     543   7296
 9807048  16823.00000  16599.63328     26 340219     42    1.33%     569   7300
 98

Uncrunching matrix
Final MIP objective                   : 1.682300000000001e+04
Final MIP bound                       : 1.682200000000000e+04
  Solution time / primaldual integral :      7611s/  5.705201%
  Number of solutions found / nodes   :        26 /  10602170
  Max primal violation      (abs/rel) : 1.599e-14 / 1.599e-14
  Max integer violation     (abs    ) : 1.332e-14

pruned centers: [ 42  19   2 104 140  36  87 147  89  81  91  37 118   9  17 143 150 142
  85  80  58  68  95  46 139  15 134  32   7  38 110 146 141  34 108  83
  51  64  48 106   8 121  78  79  26  98  75  45 116  54  93  82 145  77
  47   4  57  24 113  76  18  22 126   6  23 117 127  63 109 120  33  12
  20  13 137  67 100 105 112  11  71  66  39  49  74  73   3   5  31  16
 123 138 122  84 101  44  14  96  28  97   1 114  35 131 119  65  25 125
  10  86  40  90  21 115 103  29  55  50 136 148  69  56 132 102  52  88
  99  62 128 144 135 111  43  72  70  59  60 124  30 133  92 149 107 129
  53  41  94  61  2

#### Add a constraint that at least 80% of nodes to be taken from pruned set

In [2]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random

def kmedian_xpress_solver_constrained(points,n,c,fac_index,k):
    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])

    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])
    S=range(n)
    fac_inverse = [i for i in S if i not in fac_index]
    print(colored(fac_inverse,'magenta'))
    if len(fac_inverse) > 0:
        c4=[xp.Sum(y[fac_inverse])<=2]
    else:
        c4=[xp.Sum(y[0])<=1]
    #c4=[xp.Sum(y[fac_index])>=int(0.8*k)]

    profit=xp.Sum(xp.Sum((c*x))+xp.Sum(y)*Fac_Open_Cost)
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c2,c3,c4)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv),int(nv/2)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]

                        cost_mtr=np.zeros([n,n])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_constrained(data,n,cost_mtr,ind_last_thre,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(median_indices)+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                        s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                        s=s[0][1]
                        value_n[th_index]=value/s
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))

.................................................... 1
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 30, 31, 32, 33, 35, 36, 37, 38, 41, 43, 44, 45, 46, 47, 48, 50, 53, 56, 57, 62, 63, 65, 66, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 88, 90, 92, 94, 95, 96, 97, 99, 100, 103, 104, 105, 107, 108, 109, 111, 112, 115, 116, 117, 119, 120, 121, 122, 125, 126, 133, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 149]
FICO Xpress v8.10.0, Hyper, solve started 20:45:00, Aug 17, 2021
Heap usage: 11MB (peak 11MB, 1073KB system)
Minimizing MILP noname
Original problem has:
     22651 rows        22650 cols        67600 elements     22650 globals
Presolved problem has:
     22651 rows        22650 cols        67600 elements     22650 globals
Presolve finished in 0 seconds
Heap usage: 18MB (peak 27MB, 1074KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.

    1510  18792.00000  13948.50913     16   1173     27   25.77%     393     15
    1610  18792.00000  13948.50913     16   1283     13   25.77%     696     15
    1710  18792.00000  13948.50913     16   1341     12   25.77%     822     15
    1811  18792.00000  13948.50913     16   1402     25   25.77%     575     15
    1912  18792.00000  13948.50913     16   1425     32   25.77%     458     15
    2019  18792.00000  13948.50913     16   1494     37   25.77%     315     15
    2122  18792.00000  13948.50913     16   1554     35   25.77%     377     15
Heap usage: 1823MB (peak 1835MB, 73MB system)
B&B tree size: 15MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    2223  18792.00000  13948.50913     16   1615     28   25.77%     547     16
    2324  18792.00000  13948.50913     16   1659     40   25.77%     195     16
    2424  18792.00000  13948.50913     16   1738     10   25.77%     618     16
    2524  18792.00000  13948.50913     16   17

    6612  17839.00000  14543.54847     22   2018     18   18.47%     656     38
    6713  17839.00000  14574.74806     22   2037     24   18.30%     507     38
    6813  17839.00000  14574.74806     22   2067     23   18.30%     380     38
    6913  17839.00000  14574.74806     22   2079     21   18.30%     627     38
Heap usage: 1902MB (peak 1902MB, 77MB system)
B&B tree size: 28MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
*   6926  17791.00000  14574.74806     23   2089     21   18.08%       0     38
    7013  17791.00000  14574.74806     23   2117      9   18.08%     752     38
    7113  17791.00000  14574.74806     23   2172     20   18.08%     442     38
    7213  17791.00000  14574.74806     23   2218     27   18.08%     477     38
    7313  17791.00000  14574.74806     23   2275     29   18.08%     367     38
    7419  17791.00000  14574.74806     23   2317     25   18.08%     369     38
    7526  17791.00000  14574.74806     23   23